Lets make my own deep learning framework! ouf

In [21]:
import numpy as np

class Tensor:
    
    def __init__(self, data, autograd=False, creators=None, creation_op=None, id=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        self.autograd = autograd
        self.children = {}
        if id is None:
            id = np.random.randint(0,1000000)
        self.id = id
        
        # Add it self as a child of its creators
        if creators is not None:
            for c in creators:
                if self.id not in c.children:
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
                    
    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if cnt != 0:
                return False
        return True
        
    def backward(self, grad=None, grad_origin=None):
        if self.autograd:
            if grad_origin is not None:
                if self.children[grad_origin.id] == 0:
                    raise Exception("cannot backprop more than once!")
                else:
                    self.children[grad_origin.id] -= 1
            
            if self.grad is None:
                self.grad = grad
            else:
                self.grad += grad
            
            if self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None):
                if self.creation_op == 'add':
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                elif self.creation_op == 'neg':
                    self.creators[0].backward(self.grad.__neg__())
        
    def __add__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data + other.data, autograd=True, creators=[self,other],creation_op='add')
        
        return Tensor(self.data + other.data)
    
    def __neg__(self):
        if self.autograd:
            return Tensor(self.data * -1, autograd=True, creators=[self],creation_op='neg')
        
        return Tensor(self.data * -1)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [37]:
a = Tensor([1,2,3], autograd=True)
b = Tensor([4,2,3], autograd=True)
c = Tensor([15,6,3], autograd=True)
d = Tensor([1,0,4], autograd=True)

e = a + (-b)
f = c + (-b)
g = c + d + e + f

In [38]:
print(g.backward(Tensor([1,1,1])))


None


In [39]:
print(b.grad.data)

[-2 -2 -2]
